# Short-Circuit Calculation according to IEC 60909

pandapower supports short-circuit calculations with the method of equivalent voltage source at the fault location according to IEC 60909. The pandapower short-circuit calculation supports the following elements:

- sgen (as motor or as full converter generator)
- gen (as synchronous generator)
- ext_grid
- line
- trafo
- trafo3w
- impedance

with the correction factors as defined in IEC 60909. Loads and shunts are neglected as per standard. The pandapower switch model is fully integrated into the short-circuit calculation. 

The following short-circuit currents can be calculated:
- ikss (Initial symmetrical short-circuit current)
- ip (short-circuit current peak)
- ith (equivalent thermal short-circuit current)

either as

 - symmetrical three-phase or
 - asymmetrical two-phase
 
short circuit current. Calculations are available for meshed as well as for radial networks. ip and ith are only implemented for short circuits far from synchronous generators.

The results for all elements and different short-circuit currents are tested against commercial software to ensure that correction factors are correctly applied. 

### Example Network

Here is a little example on how to use the short-circuit calculation. First, we create a simple open ring network with 4 buses, that are connected by one transformer and two lines with one open sectioning point. The network is fed by an external grid connection at bus 1:

<img src="shortcircuit/example_network_sc.png">

In [31]:
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)
    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)
    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

## Symmetric Short-Circuit Calculation

### Maximum Short Circuit Currents
Now, we load the network and calculate the maximum short-circuit currents with the calc_sc function:

In [46]:
net = ring_network()
sc.calc_sc(net, case="max", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.262432,0.505834,0.263723
1,0.476454,0.942589,0.479039
2,0.466671,0.915418,0.469123
3,0.469892,0.924301,0.472386


where ikss is the initial short-circuit current, ip is the peak short-circuit current and ith is the thermal equivalent current.

For branches, the results are defined as the maximum current flows through that occurs for a fault at any bus in the network. The results are available seperately for lines:

In [47]:
net.res_line_sc

,ikss_ka,ip_ka,ith_ka
0,0.466671,0.915418,0.469123
1,0.459100,0.894771,0.461455
2,0.469892,0.924301,0.472386


and transformers:

In [48]:
net.res_trafo_sc

,ikss_hv_ka,ikss_lv_ka
0,0.238227,0.476454


### Minimum Short Circuit Currents

Minimum short-circuits can be calculated in the same way. However, we need to specify the end temperature of the lines after a fault as per standard first:

In [49]:
net = ring_network()
net.line["endtemp_degree"] = 80
sc.calc_sc(net, case="min", ith=True, ip=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.209946,0.462534,0.211736
1,0.384422,0.860874,0.387974
2,0.377608,0.832832,0.380846
3,0.379861,0.841982,0.383197


The branch results are now the minimum current flows through each branch:

In [50]:
net.res_line_sc

,ikss_ka,ip_ka,ith_ka
0,0.372278,0.811639,0.375302
1,0.372278,0.811639,0.375302
2,0.379861,0.841982,0.383197


In [51]:
net.res_trafo_sc

,ikss_hv_ka,ikss_lv_ka
0,0.186139,0.372278


### Asynchronous Motors

Asynchronous motors can be specified by creating a static generator of type "motor". For the short circuit impedance, an R/X ratio "rx" as well as the ratio between nominal current and short circuit current "k" has to be specified:

In [63]:
net = ring_network()
pp.create_sgen(net, 2, p_kw=0, sn_kva=500, k=1.2, rx=7., type="motor")
net

This pandapower network includes the following parameter tables:
   - ext_grid (1 elements)
   - line (3 elements)
   - switch (1 elements)
   - trafo (1 elements)
   - bus (4 elements)
   - sgen (1 elements)

If we run the short-circuit calculation again, we can see that the currents increased due to the contribution of the inverteres to the short-circuit currents.

In [64]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.263235,0.506740,0.264524
1,0.477967,0.944253,0.480546
2,0.468214,0.917129,0.470660
3,0.471344,0.925866,0.473833


### Synchronous Generators

Synchronous generators can also be considered in the short-circuit calculation with the gen element. According to the standard, the rated cosine(phi) "cos_phi", rated voltage "vn_kv", rated apparent power "sn_kva" and subtransient resistance "rdss" and reactance "xdss" are necessary to calculate the short circuit impedance:

In [66]:
net = ring_network()
pp.create_gen(net, 2, p_kw=0, vm_pu=1.0, cos_phi=0.8, vn_kv=22, sn_kva=5e3, xdss=0.2, rdss=0.005)
net

This pandapower network includes the following parameter tables:
   - ext_grid (1 elements)
   - line (3 elements)
   - gen (1 elements)
   - switch (1 elements)
   - trafo (1 elements)
   - bus (4 elements)

and run the short-circuit calculation again:

In [67]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

hp.pandapower.shortcircuit.calc_sc - WARNING: aperiodic and thermal short-circuit currents are only implemented for faults far from generators!


,ikss_ka,ip_ka,ith_ka
0,0.265319,0.513780,0.266647
1,0.482287,0.958532,0.484950
2,0.472489,0.931356,0.475018
3,0.475575,0.939648,0.478143


Once again, the short-circuit current increases due to the contribution of the generator. As can be seen in the warning, the values for peak and thermal equivalent short-circuit current will only be accurate for faults far from generators.

## Meshed Networks

The correction factors for aperiodic and thermal currents differ between meshed and radial networks. pandapower includes a meshing detection that automatically detects the meshing for each short-circuit location. Alternatively, the topology can be set to "radial" or "meshed" to circumvent the check and save calculation time.

We load the radial network and close the open sectioning point to get a closed ring network:

In [68]:
net = ring_network()
net.switch.closed = True
sc.calc_sc(net, topology="auto", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.262432,0.505834,0.263723
1,0.476454,0.942589,0.479039
2,0.470593,0.926244,0.473098
3,0.471649,0.929174,0.474168


the network is automatically detected to be meshed and application factors are applied. This can be validated by setting the topology to radial and comparing the results:

In [69]:
sc.calc_sc(net, topology="radial", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.262432,0.505834,0.263723
1,0.476454,0.942589,0.479039
2,0.470593,0.926244,0.473098
3,0.471649,0.929174,0.474168


If we look at the line results, we can see that the line currents are significantly smaller than the bus currents:

In [70]:
sc.calc_sc(net, topology="auto", ip=True, ith=True)
net.res_line_sc

,ikss_ka,ip_ka,ith_ka
0,0.279812,0.550740,0.281301
1,0.190781,0.375504,0.191796
2,0.344176,0.678046,0.346014


this is because the short-circuit current is split up on both paths of the ring, which is correctly considered by pandapower.

## Fault Impedance

It is also possible to specify a fault impedance in the short-circuit calculationn:

In [71]:
net = ring_network()
sc.calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.)

which of course decreases the short-circuit currents:

In [72]:
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.261343,0.503509,0.262627
1,0.469382,0.926656,0.471909
2,0.459875,0.900379,0.462274
3,0.463005,0.908972,0.465446


## Asymetrical Two-Phase Short-Circuit Calculation

All calculations above can be carried out for a two-phase short-circuit current in the same way by specifying "2ph" in the fault parameter:

In [75]:
net = ring_network()
sc.calc_sc(net, fault="2ph", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.227273,0.438065,0.228391
1,0.412621,0.816306,0.414860
2,0.404149,0.792775,0.406272
3,0.406938,0.800468,0.409099


Two phase short-circuits are often used for minimum short-circuit calculations:

In [76]:
net = ring_network()
net.line["endtemp_degree"] = 150
sc.calc_sc(net, fault="2ph", case="min", ip=True, ith=True)
net.res_bus_sc

,ikss_ka,ip_ka,ith_ka
0,0.181818,0.400566,0.183369
1,0.332920,0.745538,0.335995
2,0.326772,0.717815,0.329521
3,0.328807,0.726834,0.331657
